#### 将原来6G的xml文件切分成平均100k的小文档

In [2]:
RUN = 0
if(RUN):  
    fin = open('zhwiki-20171020-pages-articles-multistream.xml',encoding='utf8')
    for i in range(10000000):      
        readin = fin.readlines(100*1000)
        if(not readin):
            break
        fout = open('parts/part_'+str(i),'w',encoding = 'utf8')
        fout.writelines(readin)
        # fout.writelines(re.sub("[\[\]\'\"\@#$%^&*()<>：{}|/_.+-—，,。!！ ？、~@#￥%……&*（）]+", "",s))
        fout.close()
    fin.close()
    partsNum = i
    print(partsNum)
else:
    partsNum = 49125
    

#### 导入词典
词典选自https://github.com/fxsjy/jieba/tree/master/extra_dict

In [3]:
word2int = {}
idx = 0
with open('dict/dict',encoding='utf8') as fin:
    for line in fin.readlines():
        word = line.split()[0]
        if(word not in word2int):
            word2int[word] = idx
            idx += 1
wordsNum = len(word2int)
print('size of dict: ',len(word2int))

with open('dict/dict_unique','w',encoding='utf8') as fout:
    for i in word2int:
        fout.write(i+'\n')

size of dict:  583280


#### 做位运算的集合BitSet

In [4]:
class BitSet:
    def __init__(self, elementNum):
        self.bytesNum = round(elementNum/8)
        self.set = bytearray(self.bytesNum)
        
    def insert(self, index):
        if(index >= 0 and index>>3 < self.bytesNum):
            self.set[index >> 3] = self.set[index >> 3] | (1 << int(index & 7)) 
            return True;
        return False;
    
    def isElement(self, index):
        if(index >= 0 and index>>3 < self.bytesNum and (self.set[index >> 3] & (1<<(index & 7)))):
            return True
        return False
        
    def AND(self, bs):
        a = BitSet(self.bytesNum*8)
        for i in range(self.bytesNum):
            a.set[i] = self.set[i] & bs.set[i]
        return a
    
import sys,time
# 显示进度条
def processBar(index, totalNum):
    bar = '#'*int((index/totalNum)*50)
    sys.stdout.write(str(int((index/totalNum)*100))+'%  ||'+bar+'->'+str(index)+'/'+str(totalNum)+"\r")
    #sys.stdout.flush()


#### jieba 为分词工具

In [5]:
import jieba
jieba.load_userdict('dict/dict_unique')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELLT5~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.098 seconds.
Prefix dict has been built succesfully.


#### 给所有词语构建倒排索引

In [6]:
partsNum = 49125

RUN_iidx = 0
if(RUN_iidx):
    
    invertedIndex = [BitSet(partsNum) for i in range(wordsNum)]
    print('每个词语的倒排索引表的字节数：',invertedIndex[0].bytesNum)
    
    for partIdx in range(partsNum):

        processBar(partIdx, partsNum)

        with open('parts/part_'+str(partIdx),encoding='utf8') as f:
            content = f.read()
            for word in jieba.cut_for_search(content):
                try:
                    #if(is_ch(word[0])):
                    invertedIndex[word2int[word]].insert(partIdx)
                except KeyError as err:
                    #print('KEY ERROR'+str(err))
                    pass

    for idx in range(wordsNum):    
        processBar(idx, wordsNum)
        with open('iidx/ch_'+str(idx)+'.iidx','w',encoding='utf8') as f:
                for j in invertedIndex[idx].set:
                    f.write(str(j)+' ')

In [7]:
def readIidx(file):
    with open(file,encoding='utf8') as f:
        s = f.read()[:-1].split(' ')
    bSet = BitSet(len(s)*8)
    for i in range(bSet.bytesNum):
        bSet.set[i] = int(s[i])
    return bSet


#### 繁简转换测试

In [8]:
from langconv import *

line = '為当今世界著名的[[美國]][[網球]]女子巨星，網球史上最伟大的女子选手之一，\
    美國體育史上最佳女子網球选手（與[[克里斯·埃弗特|Chris Evert]]並列），\
    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，\
    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。'
# 转换繁体到简体
line = Converter('zh-hans').convert(line)
print(line)
# 转换简体到繁体
line = Converter('zh-hant').convert(line)
print(line)

为当今世界著名的[[美国]][[网球]]女子巨星，网球史上最伟大的女子选手之一，    美国体育史上最佳女子网球选手（与[[克里斯·埃弗特|Chris Evert]]并列），    全世界第一位网球大满贯黑人女单冠军，单打最高世界排名第一，    23座大满贯女单冠军得主，网球史上第二位女子单打生涯全满贯得主。
為當今世界著名的[[美國]][[網球]]女子巨星，網球史上最偉大的女子選手之一，    美國體育史上最佳女子網球選手（與[[克裡斯·埃弗特|Chris Evert]]併列），    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。


#### 找出包含问题中关键词的文档

In [9]:
def mask(bytes):
    r = BitSet(bytes*8)
    for i in range(bytes):
        r.set[i] = 255
    return r

bsMask = mask(round(partsNum/8)) # 掩码

In [10]:
stopWords = ['上','的','什么','什麼','是','大','第几',\
             '过','多少','多少次','？','哪','谁','哪里',\
            '哪个','哪种','哪位','哪座','指','和']

def findDoc(query):
    keyList = []
    targets = bsMask
    for word in jieba.cut_for_search(query):
        if(word not in stopWords):
            try:
                #print(word)
                keyList.append(word2int[word])
                targets = targets.AND(readIidx('iidx/ch_'+str(word2int[word])+'.iidx'))
            except KeyError:
                pass
    result = []
    for i in range(partsNum):
        if(targets.isElement(i)):
            result.append(i)  
    #print('doc number: ',len(result))
    return result[:10]

#### 找到包含关键词最多的语段

查询的时候分别用简体繁体查了一遍

In [11]:
def findLine(query, minLen=10, maxReturn=1):
    s = []
    docList = findDoc(query)
    keywords = '.'.join(jieba.cut_for_search(query)).split('.')
    #keywords = '.'.join(jieba.cut(query,cut_all=False)).split('.')


    for docIdx in docList:
        with open('parts/part_'+str(docIdx),encoding='utf8') as f:
            for line in f.readlines():
                if(len(line) > minLen):
                    tf = {}
                    for word in jieba.cut_for_search(line):
                        if(word in keywords and word not in stopWords):
                            tf[word] = 1 
                    bingo = len(tf)
                    s.append(('位于part_'+str(docIdx)+'   '+line,bingo))
                    
    query = Converter('zh-hant').convert(query)
    docList = findDoc(query)
    keywords = '.'.join(jieba.cut_for_search(query)).split('.')
    #keywords = '.'.join(jieba.cut(query,cut_all=False)).split('.')
    #print(keywords)

    for docIdx in docList:
        with open('parts/part_'+str(docIdx),encoding='utf8') as f:
            for line in f.readlines():
                if(len(line) > minLen):
                    tf = {}
                    for word in jieba.cut_for_search(line):
                        if(word in keywords and word not in stopWords):
                            tf[word] = 1 
                    bingo = len(tf)
                    s.append(('位于part_'+str(docIdx)+'   '+line,bingo))
    s = sorted(s,key=lambda x:x[1], reverse=1)
    return [item[0] for item in s[:maxReturn]]

#### 查询

In [12]:
query = '西周时期音阶宫，商，角，徵'
findLine(query,maxReturn=3)

['位于part_321   西周音樂的發展，表現在樂器種類的增多和音樂理論的發展。西周結合禮儀制度與音樂和舞蹈，後世通稱[[雅樂]]。雅樂的黃金時代從西周到春秋早期，在春秋晚期走向衰退。禮樂的規模按照貴族等級，有嚴格的大小規模：周王的樂隊可以四面排列，為宮懸。諸侯則三面排列，為軒懸。卿和大夫則兩面排列，為判懸。士只有一面，為特懸。懸為原指樂懸，又可代稱整個樂隊&lt;ref&gt;《周禮·春官宗伯》：「正樂縣之位，王宮縣，諸侯軒縣，卿大夫判縣，士特縣，辨其聲。」&lt;/ref&gt;。據說舞蹈團的規模也有規範：周王用八佾，即八八六十四名舞者。諸侯為六佾，即八六四十八名舞者，大夫為四佾，即八四三十二名舞者、士為兩佾，即八二十六名舞者&lt;ref&gt;《穀梁傳·隱公五年》：「即謂天子八佾，諸公六佾，諸侯四佾之等也。」&lt;/ref&gt;。曲目也有限制，周王祭祖可用的《詩經·周頌·雍》，士大夫不可使用。周室使用雅樂的時機有用於祭祀（郊禮、社禮）、饗宴、射禮、戰勝慶典之禮（王師大獻）、行軍田役等等。音樂包含六代樂舞、小舞、散樂、四夷之樂、房中樂、詩樂及其他宗教性樂舞（如求雨的舞雩和驅除疫鬼的[[儺]]）。當時的樂官為大司樂，負責音樂禮儀管理和教育&lt;ref&gt;《周禮·春官宗伯》：「大司樂：掌成均之法，以治建國之學政，而合國之子弟焉。......以六律、六同、五聲、八音、六舞大合樂。」&lt;/ref&gt;。周朝貴族與國人普遍重視音樂修養，與禮儀深深綁在一起。東周之後，禮樂敗壞，下位者僭越上位者的禮儀。例如執政魯國的仲孫、叔孫、季孫等三家大夫用周王在用的《詩經·周頌·雍》來結束祭祀&lt;ref&gt;《論語·八佾》：三家者以雍徹。子曰：「『相維辟公，天子穆穆』，奚取於三家之堂？」&lt;/ref&gt;。另一方面，雅樂瓦解，樂師各奔前程。例如太師摯投奔齊國、亞飯-{干}-投奔楚國、三飯缭投奔蔡國、四飯缺投奔秦國&lt;ref&gt;《論語·微子》：「大師摯適齊，亞飯干適楚，三飯繚適蔡，四飯缺適秦。鼓方叔入於河，播鼗武入於漢，少師陽、擊磬襄，入於海。」&lt;/ref&gt;。周朝樂器種類繁多，當時的樂器除編鐘、編磐和大小不同的鼓等打擊樂器之外，像琴、瑟等弦樂器，笙、竽等管樂器，也都出現。所謂金、石、絲、竹、瓠、革之音，大都齊備。樂器增多，必須注意演奏的

#### 训练样本

In [13]:
questions = []
with open('wdm_assignment_3_samples.txt',encoding='utf8') as fin:
    for line in fin.readlines():
        questions.append(line.split('\t')[0])

questions[:10]

['马尔代夫的第一大支柱产业是什么？',
 '澳大利亚是南半球面积第几大的国家？',
 '阿根廷国家足球队赢得过多少次美洲杯冠军？',
 '平均人口密度最高的大洲是？',
 '中国历史上最杰出的浪漫主义诗人是？',
 '甲午战争爆发的标志是？',
 '《华英字典》的作者是？',
 '《资治通鉴》的撰写一共耗时多少年？',
 '世界上首位从南坡登上珠穆朗玛峰的女性的国籍是？',
 '最后宣布废除奴隶制度的行政地区是？']

#### 样本问答测试

In [246]:
for q in questions:
    print('问:',q)
    print('答:',findLine(q,maxReturn=1))
    print()

问: 马尔代夫的第一大支柱产业是什么？
答: ['位于part_630   马尔代夫是个岛国，陆地面积相当狭窄。[[工业]]、[[农业]]水平低下。[[渔业]]、[[航运]]和[[旅游]]是三大经济支柱。近年来，随着旅游业的发展，旅游收入已达到了国民总产值的70%以上。成为马尔代夫的第一支柱产业。\n']

问: 澳大利亚是南半球面积第几大的国家？
答: ["位于part_116   = 1-876429-37-2|date=2003}}&lt;/ref&gt;&lt;ref&gt;{{cite web |title = Australia |work=Oxford Dictionaries |date=April 2010 |publisher=Oxford University Press |url = http://oxforddictionaries.com/definition/english/Australia |accessdate=2012-07-26}}&lt;/ref&gt;，[[缩写]]为'''AU'''、'''AUS'''），中文环境下（尤其是臺灣、香港等地）常使用「'''澳-{}-洲'''」代替“-{zh-cn:澳大利亚;zh-tw:澳大利亞}-”，是全球[[面积]]第六大的[[国家]]，[[大洋洲]]最大的国家和[[南半球]]第二大的国家。澳大利亚国土包括澳大利亚大陆，[[塔斯曼尼亚|塔斯曼尼亚岛]]及数个海外的岛屿，面积和[[美國本土]]相似，是[[世界]]上唯一一个国土覆盖整个[[大陆]]的国家。與隔海相望的东南近邻是[[紐西蘭]]，西北是[[印度尼西亚]]，北边是[[巴布亚新几内亚]]、[[西巴布亚]]及[[东帝汶]]。\n"]

问: 阿根廷国家足球队赢得过多少次美洲杯冠军？
答: ['位于part_3667   到了1980年代，“王子”[[恩佐·弗朗西斯科利|弗朗西斯科利]]、“灵猫”[[鲁本·索萨|索萨]]、本戈切亚、丰塞卡、波耶特等一批球星的出现使乌拉圭足球有了转机。1980年底到1981年初，为纪念世界杯创办50周年，乌拉圭举办了[[1980-81年世界冠軍金盃足球賽|世界金杯足球赛]]，邀请历届世界杯冠军得主齐聚一堂，这被认为是后来[[联合会杯]]的雏形。意大利、西德、巴西、阿根廷应邀参赛，惟英格兰拒绝参加，改由兩届世

答: ["位于part_489   '''直立人'''又稱'''直立猿人'''，其生存年代为更新世早期至中期。直立人已经能够直立行走并且制造石器，是舊石器時代早期的人類。[[北京人]]、[[蓝田人]]、[[元謀人]]、[[巫山人]]等都属于直立人。它仍带有猿类特徵，如头盖骨低平，眉骨粗壮，吻部前伸。但也有現代人特徵，如可雙足直立和腦容量比猿類大。[[欧仁·杜布瓦]]於1891年在[[爪哇岛]][[梭罗河]]附近发现的[[爪哇人]]化石，[[裴文中]]於1923年在[[中國]][[北京]]附近的[[周口店]]所發現的[[北京猿人|北京人]]化石，還有[[藍田人]]、[[巫山人]]和[[元謀人]]等，都是直立人的例子。\n"]

问: 美国的当代人类学通常划分为几大分支？
答: ['位于part_822   [[台灣原住民|臺灣原住民族]]屬於[[南島民族]]，目前約五十多萬人。依據人類學者的研究分析，原住民的[[屋架建築]]、[[火墾]]、食[[檳榔]]、[[紋面]]、皮衣製作、[[輪舞]]等文化習俗，都與傳統的南島文化相近。過去大多數的原住民都是以傳統[[游耕]]及[[狩獵]]為主要的生產方式，近年來靠近平地的原住民則與[[漢人]]的生活方式接近，但是部落組織的維繫仍然保留著。親族制度中，[[泰雅族]]、[[布農族]]為父系社會；[[阿美族]]、[[卑南族]]則為母系社會。部落組織中，[[魯凱族]]與[[排灣族]]為貴族社會；[[賽夏族]]、[[布農族]]則為氏族社會。\n']

问: “中华民族”一词最早由谁提出？
答: ['位于part_73   「中華民族」的概念最早由[[梁启超]]於1902年提出，以激發[[中國民族主義]]&lt;ref name=&quot;首提&quot;&gt;{{cite web|url=http://www.confuchina.com/08%20xiandaihua/zhonghuaminzu.htm|title=「中華民族」是誰首提的？|accessdate=|author=|date=|publisher=李喜所|language=zh}}&lt;/ref&gt;。[[辛亥革命]]之後，由於[[五族共和]]與[[同化|民族融合]]的口號廣為流傳以及隨著諸多學者的提倡，使得「中華民族」而廣為人知&lt;ref&gt;林家有，廣

答: ['位于part_6   這些核心理論大致包括於[[經典力學]]、[[量子力學]]、[[熱力學]]、[[統計力學]]、[[電磁學]]、[[狹義相對論]]等等基礎物理學領域，是進階研究專門論題的重要工具。&lt;ref&gt;{{cite web\n']

问: 到2012年为止，联合国共有多少个成员国？
答: ['位于part_80   到2012年為止，聯合國共[[聯合國成員國列表|193個成員國]]，包括除[[梵蒂岡城國]]以外所有无争议的[[主權國家]]都是联合国会员国。&lt;ref&gt;{{cite web |url = https://www.un.org/depts/dhl/unms/whatisms.shtml | title=What are Member States? |publisher=聯合國官方網站 |accessdate=2010-12-05}}&lt;/ref&gt;在聯合國遍及世界的辦事處中，聯合國及其專門機構通過全年舉行定期會議來決定實體和行政議題。聯合國由六大主要機構組成：[[聯合國大會]]（主要的[[審議會|審議機構]]）、[[聯合國安全理事會|安理會]]（以決定對和平與安全的某些決議）、[[聯合國經濟及社會理事會|經濟及社會理事會]]（以協助促進國際經濟和社會的合作和發展）、[[聯合國秘書處|秘書處]]（為聯合國提供所需的研究、資訊和設施）、[[國際法院]]（主要的司法機構）以及[[聯合國託管理事會]]（當前不活躍）。其他重要的[[聯合國機構]]還有[[世界衛生組織]]、[[世界糧食計畫署]]和[[聯合國兒童基金會]]。聯合國的行政首長是[[聯合國秘書長]]。聯合國的經費由會員國分攤和自願捐贈。聯合國現在有六種工作语言，分別為：[[阿拉伯语]]、[[汉语]]、[[英语]]、[[法语]]、[[俄语]]、[[西班牙语]]&lt;ref name=&quot;langs&quot;&gt;{{cite web |url = https://www.un.org/zh/aboutun/languages.shtml |title=联合国一览-联合国官方语文 |publisher= 聯合國 |accessdate=2013年10月8日 }}&lt;/ref&gt;。[[聯合國中文日]]為每年的4月20日。\n']

问: 北京

答: ['位于part_32   [[洛阳]]（十三朝古都）、[[开封]]（七朝古都）、[[安阳]]（七朝古都）、以及[[郑州]]（五朝古都）都是著名的千年古都&lt;ref &gt;{{cite web|url=http://culture.hebei.com.cn/system/2011/04/29/011111897.shtml | language= | title=七朝古都——安阳 |publisher=长城网 |date=2011-04-29 |accessdate= }}&lt;/ref&gt;。河南在国务院颁布的109个历史文化名城中占据了8个，居全国第二。[[夏代]]在偃师[[二里头]]（今河南洛阳偃师）建都，后迁都[[阳城]]（今河南郑州登封），后迁都[[阳翟]]（今河南许昌禹州），[[商代]]在亳（今河南商丘附近）建都，后来迁到“殷”（[[安阳]]），遂称[[殷商]]。以后许多朝代在[[洛阳]]建都。每当中国内乱时期，群雄“逐鹿中原”，[[河南]]是必争之地，“[[中国]]”的名称就来源于中心位于[[中原]]地带的国家。\n']

问: 1916年至1927年，北京大学的校长是？
答: ['位于part_67   2004年至2006年，李敖与[[凤凰卫视]]合作的节目《[[李敖有话说]]》面向[[香港]]与[[中国大陆]]，并促成他于2005年访问中国大陆。2005年9月19日，李敖展开「[[李敖神州文化之旅|神州文化之旅]]」，行程包括[[香港]]、[[北京]]和[[上海]]。过程中，分别于[[北京大学]]、[[清华大学]]和[[复旦大学]]进行了演讲，他主張演讲的目的不在於政治，而是文化與思想的活動&lt;ref&gt;{{Cite web |url = https://www.youtube.com/watch?v=Gd4ZBj3yFrk |title = 五十六年家园─李敖神州文化之旅全纪录_4.中西文化 | }}&lt;/ref&gt;&lt;ref&gt;{{Cite web |url = https://www.youtube.com/watch?v=1vGdBElSN7M |title = \n']

问: 江南三大名湖分别是？ 西湖、玄武湖、南湖

答: ["位于part_851   '''玄武湖'''位于[[江苏]][[南

答: ['位于part_3   [[西汉]]时[[司马迁]]撰写了《[[史记]]》，创建了[[纪传体]]的历史记录体裁，《史记》的规模在当时世界范围内是空前的。之后[[东汉]][[班固]]著《汉书》，延续发展了《史记》的体例，是中国第一部纪传体[[断代史]]。这两部历史著作，奠定了中国古典史学的基础，后来的历史学家沿用《史记》和《汉书》的体裁，将各个[[朝代]]的历史汇编成书，组成了“[[二十四史]]”。除断代史之外，[[唐]][[宋朝|宋]]期间中国还出现了[[通史]]，如唐末[[杜佑]]的《[[通典]]》，宋[[司马光]]的《[[资治通鉴]]》，其中《资治通鉴》是叙事长达一千三百六十二年的[[编年体]]通史，是中国史学史上的奇葩。&lt;ref name=&quot;zbjn1&quot;/&gt;\n']

问: 世界第二高峰的海拔是？
答: ['位于part_35   以[[龙门山]]-大凉山一线为界，四川大致可分为东部四川盆地及盆缘山地区，以及西部川西高山高原及川西南山地区。四川地形复杂多样，包含[[四川盆地]]、川西高原（青藏高原）、[[横断山脉]]、[[云贵高原]]、[[秦岭]]-[[大巴山]]山地等几大地貌单元，地跨中国地势第一及第二阶梯，西高东低，由西北向东南倾斜，相对高差达7300多米。全省最高点是川西的[[大雪山]]主峰[[贡嘎山]]，海拔高达7556米，超过除亚洲外[[各洲最高峰|世界其它各大洲最高峰]]。\n']

问: 单片机的最大优点是？
答: ["位于part_832   '''-{zh-cn:单片机; zh-tw:單晶片}-'''，全称'''-{zh-cn:单片微型计算机; zh-tw:單晶片微電腦}-'''（{{lang-en|'''single-chip microcomputer'''}}），又稱'''-{微控制器}-'''（{{lang|en|'''microcontroller'''}}），是把[[中央处理器]]、[[電腦記憶體|存储器]]、定时/计数器（{{lang|en|timer/counter}}）、各種[[I/O|输入输出接口]]等都集成在一块[[集成电路|-{zh-hans:集成电路; zh-hant:積體電路;}-]][[芯片]]上的[[微型计算机]]。与应用在个人电脑中的通用型[[微处理器]]相比，它更强调

答: []

问: 牛津大学培养了几位获得过菲尔兹奖的数学家？
答: ['位于part_390   学校现有15个教学系（院、部），35个本科专业，基本形成了适应区域经济社会发展需要，以工学为主体，文学、管理学为两翼，理、工、文、管、法、艺多学科协调发展的学科专业布局。拥有教育部专业综合改革试点项目1个，教育部大学生校外实践教育基地1个，教育部特色专业建设点1个；省级特色专业3个，省级专业综合改革试点项目2个，省级实践教学示范中心2个，省级校企合作人才培养示范基地2个，省级基础课示范实验室1个，省级大学生创新训练中心1个，省级优秀实习教学基地10个，省级精品课程7门，省级教学团队2个。学校已形成拥有4个省级&quot;十二五&quot;重点建设学科、2个湖南省社会科学研究基地、2个湖南省高校科技创新团队、1个湖南省高校产学研合作示范基地、1个长沙市文化研究基地、8个校级重点学科、3个校级重点建设学科、3个校级科研创新公共平台、9个校级创新团队及团队培育对象、20个校级研究所的学科建设格局。\n']

问: 世界四大长河是？
答: ["位于part_243   '''洛阳市'''（{{Lang-pinyin|'''Luòyáng'''}}，[[邮政式拼音]]：'''Loyang'''）是[[中华人民共和国]][[河南省]]下辖的[[地级市]]，位于[[中原]]地区，中国[[四大古都]]之一，现为河南省及[[中原城市群]]的副中心城市。是[[丝绸之路]]起点，[[隋唐大运河]]的中心城市，也是[[中华人民共和国国务院|国务院]]认定的首批24个[[国家历史文化名城]]之一，2011年被[[联合国]]授予“世界文化名城”称号&lt;ref name=&quot;sjmc&quot;/&gt;。洛阳自夏朝开始，前后有十三个正统朝代以其为都，是中国历史上建都最早、建都政权数量最多、持续建都时间最久的都城，拥有5000年文明史、3000年建城史和1500年建都史。仰韶曙光，夏商肇基，周制礼乐，两汉文章，魏晋风流，南北融合，隋唐盛世，宋金兵戈，洛阳城在历史的长河中数度兴废，活跃在文明中心长达数千年，与国家与民族的命运始终相依。&lt;ref&gt;{{cite web|url=http://www.hnjjbs.com/Recommend/2010-12-10/3134.html

答: ['位于part_1377   現代培植的香蕉会因感染[[黃葉病]]而引發[[細胞程序性死亡]]。\n']

问: 目前市面上流通的主要人民币是第几套？
答: []

问: 中国气温最低的县份是？
答: ['位于part_492   漠河县在中国最北端毗邻俄国西伯利亚地区，地处寒[[温带季风气候]]，天气严寒而且常年温度较低。冬季气温一般在-40℃以下，是全中国气温最低的县份。年平均气温为-5.5℃；平均气温零度以下日子达8个月；平均无霜期只有86.2日；年平均降水量431.0毫米；而全年主风向为西北风。漠河县一月份平均温度-30.9°C，冬天最低气温仅-52.3°C（1969年2月13日，中国现有气象资料中的极端最低气温记录）。七月份平均温度为18.4°C，最高温可达39.3°C。&lt;ref&gt; {{en}} [http://www.mherrera.org/temp.htm Extreme Temperatures Around the World - world highest lowest temperatures]. &lt;/ref&gt;\n']

问: 中国人口最多的少数民族是？
答: ['位于part_19   辛亥革命推翻满清统治以后，革命黨在初期主张「排满」和獨立成為单一的汉民族国家，立宪派则主张「合满」和建立包括五族在内的多民族国家&lt;ref&gt;《政體進化論》&lt;/ref&gt;，并为此与革命派展开过激烈论战，最後通过《中華民國憲法》建立一個各族平等的国家。[[中華人民共和國]]成立後，[[中国共产党]]对[[中国少数民族|少数民族]]实行扶助政策，并设置了400多个[[民族区域自治制度|民族自治区域]]。到了20世纪中期，历史学家[[翦伯赞]]針對中国和中华民族的问题，指出凡活动于今日中国境内的古代民族都算[[中国人]]；凡出现在中国历史上的民族，都是多民族国家的一个成员。&lt;ref&gt;翁有为，1999，《当代中国学术思想史》，河南大学出版社&lt;/ref&gt;按照中华人民共和国官方的划分，中国目前有[[汉族]]、[[藏族]]、[[满族]]、[[蒙古族]]、[[维吾尔族]]、[[回族]]、[[壮族]]等56个民族，汉族是人數最多的民族。汉族广泛分布在全国各地，其中主要集中在东北、华北、华东、中南、[[甘

答: []

问: 中国是哪一年加入世界贸易组织WTO的
答: ["位于part_6974   '''单独关税区'''（separate customs territory，独立关税区、单独关境、個別關稅領域）为[[關稅暨貿易總協定|关税暨贸易总协定]]('''GATT''')及其继承者[[世界贸易组织]]('''WTO''')相关术语，为对其[[缔约方]]（即成员）的总称，因缔约方属政府组织，以主权国家政府为主，包括非主权的[[政治实体]]政府。\n"]

问: 金庸小说《天龙八部》中，木婉清的母亲是谁
答: ['位于part_1577   在[[金庸]]小說《[[天龍八部 (小說)|天龍八部]]》中，段正明是[[北宋]]年間[[雲南]][[大理國]]皇帝，帝號為保定帝。\n']

问: 棕色人种生活在哪块大陆
答: ['位于part_1300   事實上，林媽利的結論存在一些根本上的問題，其一是漢人在大陸的南北本來就存在[[遺傳因子]]上的差異性，其二是[[台灣原住民]]和大陸東南閩粵等省的原住民，本具有血緣關係(台灣原住民的先祖，在冰河時期，由大陸遷徙到台灣)。基本上，生活在中國大陸的南方人(在北方漢人大量南遷前)、東南亞人及台灣原住民，在[[遺傳因子]]同屬一個相近的族群。他們都可被歸類為南方蒙古人種，而有別於今日尚存於大洋洲的[[棕色人種]](老亞洲人)。這些南方蒙古人種的族群，在[[人類Y染色體DNA單倍型類群]]上，主要屬於O1及O2兩個單倍型類，有別於北方蒙古人種(如北方漢人與藏族)的O3單倍型類。另外，根據蔡麗琴等[[遺傳因子]]專家在一份2001年發表的論文(Tsai 2001&lt;ref name=&quot;springerlink.com&quot;&gt;Li-Chin Tsai 2001, [http://www.springerlink.com/content/ayewcve71fu0mgmw/ Haplotype frequencies of nine Y-chromosome STR loci in the Taiwanese Han population ]&lt;/ref&gt;)中，對台灣漢人人口的[[Y染色體]]的研究，台灣漢人人口的[[人類Y染色體DNA單倍型類群]]組成比例為，Y染色體DNA單倍型類群 O1 22.4

答: ['位于part_625   2014年，杭州《政府工作報告》中提出，在2014年將推進[[富陽]]、臨安[[撤市建區]]工作。2015年2月，除富阳外，临安撤市并区也正在进行中。张鸿铭指出，积极推动城市群和都市圈合作发展，“力争两区尽早获批”。从杭州民政局得知，临安撤市设区并未上报，临安要撤市设区至少在要等2年时间。2015年3月25日，临安市列入列入国家发改委中小城市综合改革试点，昌化镇列为重点试点单位。2015年5月12日，临安市入选2015年中国县域经济发展潜力百强县市，排名全国73位。中国创新强县排名16名。\n']

问: 成语“明修栈道”下一句是什么
答: ['位于part_1560   项羽分封诸侯，不足一年，齐国发生内乱，项羽于是亲率楚军北上镇压。前206年八月，刘邦趁此良机进军关中，韩信说完汉中对后，刘邦遂听信计”。劉邦依計出兵结果受阻陈仓， 后刘邦采用赵衍计策绕路大败章邯；汉军大胜，旋即攻占咸阳，关中大部分归顺汉王刘邦。 （须注意：《史记》等正史均没有提及“明修栈道”一事。《史记》高祖功臣侯者年表却有这样的记录：：须昌侯赵衍，“以谒者汉王元年初起汉中。雍军塞陈（雍王章邯派兵塞陈仓道。），谒上，上计欲还。衍言从他道，道通）\n']

问: 诗人戴望舒的《雨巷》中，用什么花比喻那个他想遇到的姑娘
答: []

问: “宝带桥”是谁建造的
答: ['位于part_78   苏州现简称-{蘇（苏）}-。在夏代之前有一位很有名望的谋臣叫“胥”。他不仅有才学，而且精通天文地理，因帮助大禹治水有功，深受[[舜]]王敬重，封他为大臣，并把江东册封给胥。从此，江东便有了“姑胥”之称。在苏州护城河上即有一桥名为“姑胥桥”。胥在江东的灵岩山脚下，建造了“姑胥城”（位置在今天的苏州市吴中区木渎镇）。“姑”是当时[[古吴语]]的拟声词，无义。\n']

问: 胆汁是由人体的什么内脏器官分泌出来的
答: ['位于part_1740   肝臟是存在於所有[[脊椎動物]]，是最大的內臟器官。不同的[[物種]]，形式有很大的不同。然而，在大多數物種它分為左，右葉，但有例外，像是[[蛇]]的身體，肝臟就像是雪茄形狀。\n']

问: MPEG是什么的缩写
答: ['位于part_0   &lt;li&gt;17:55 2004年3月27日 [[User:Shizhao|Sh

答: []

问: 北宋“三苏”中，苏洵与苏轼的亲戚关系是什么
答: ['位于part_11974   蘇洵有子女六人，長子與三名女兒皆早卒。&lt;ref&gt;[[歐陽修]]《蘇明允墓誌銘》講：蘇洵“生三子：曰景先，早卒；軾，今為殿中臣直[[史館]]；轍，權大名府[[推官]]。三女皆早卒。”&lt;/ref&gt;。三女[[蘇八娘]]較蘇軾年長一歲，幼而好學，十六歲嫁表兄[[程之才]]，兩年後被夫家虐待至死&lt;ref&gt;蘇洵《自尤〈并叙〉》：「盖壬辰之岁而丧幼女，始将以尤其夫家，而卒以自尤也。女幼而好学，慷慨有过人之节，为文亦往往有可喜。既适其母之兄程浚之子之才，年十有八而死。而浚本儒者，然内行有所不谨，而其妻子尤好为无法。吾女介乎其间，因为其家之所不悦。适会其病，其夫与其舅姑遂不之视而急弃之，使至于死。始其死时，余怨之，虽尤吾之人亦不直浚。独余友发闻而深悲之，曰：“夫彼何足尤者！子自知其贤，而不择以予人，咎则在子，而尚谁怨？”予闻其言而深悲之。其后八年，而予乃作自尤诗。」 &lt;/ref&gt;&lt;ref&gt;蘇軾《乳母任氏墓誌銘》：“乳亡姊八娘及軾」&lt;/ref&gt;。[[清朝]][[袁枚]]《隨園詩話》則誤把她和蘇軾堂妹、柳仲遠之妻誤為蘇軾之妹&lt;ref&gt;袁枚《隨園詩話》：“東坡止有二妹；一適柳，一適程也。今俗傳為秦少游之妻，誤矣！”&lt;/ref&gt;。现在据考证苏小妹仅仅是一位虚构的人物，“苏小妹三难新郎”只是民間故事&lt;ref&gt;秦觀在《徐君主簿行狀》中說：潭州（今湖南长沙）宁台[[主薄]]徐成甫有“女三人，曰文美、文英、文柔……以文美妻余。”（《淮海集》卷三十六），換句話說，秦觀妻子是徐文美。&lt;/ref&gt;，也排除了是苏轼的姐姐的可能。\n']

问: 在《魔戒》系列电影中，佩戴着戒指一路披荆斩棘的矮小名族被称为什么人
答: []

问: 在《三国演义》中，桃园三结义前，刘备是以卖什么为生的
答: []

问: 毛泽东诞生于哪年，于哪年去世
答: ['位于part_89   1945年[[日本投降]]后，学校开始修缮清华园。次年联大结束，三校复员。清华在战争伤痕下努力恢复发展，1946年2月，部分系所勉強於北平清華園重新復課。5月4日，西南聯大結束，學生依志願分發，入清華者932人，入北大者64